# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(name="Getting_Started_HW").getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 17:39:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [4]:
pga_tour_data = spark.read.csv('/Users/apple/Desktop/Learning_PySpark/pga_tour_historical.csv',inferSchema=True, header=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [5]:
pga_tour_data.limit(5).show()

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+



In [6]:
pga_tour_data.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [9]:
pga_tour_data.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



In [40]:
pga_tour_data.summary()

DataFrame[summary: string, Player Name: string, Season: string, Statistic: string, Variable: string, Value: string]

In [42]:
pga_tour_data.columns

['Player Name', 'Season', 'Statistic', 'Variable', 'Value']

In [45]:
pga_tour_data.summary().toPandas()

,summary,Player Name,Season,Statistic,Variable,Value
0,count,2740404,2740403,2740404,2740404,1657247
1,mean,None,2013.973479083186,None,None,12494.388998743096
2,stddev,None,2.607050115517948,None,None,157274.75673570728
3,min,A.J. McInerney,2010,% of Potential Pts won - FedExCup Playoffs,% of Potential Pts won - FedExCup Playoffs - (...,-178
4,25%,None,2012,None,None,24
5,50%,None,2014,None,None,71
6,75%,None,2016,None,None,101
7,max,Zihao Chen,2018,World Money List,World Money List - (MONEY),3564954


## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [10]:
from pyspark.sql.types import *

In [11]:
data_schema = [ StructField("Player Name", StringType(), True),
               StructField("Season", IntegerType(), True),
               StructField("Statistic", StringType(), True),
               StructField("Variable", StringType(), True),
               StructField("Value", IntegerType(), True),
              ]
final_structure = StructType(fields=data_schema)

In [13]:
pga_tour_data = spark.read.csv('/Users/apple/Desktop/Learning_PySpark/pga_tour_historical.csv', schema=final_structure)
pga_tour_data.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Player Name,NaN,Statistic,Variable,NaN
1,Robert Garrigus,2010.0,Driving Distance,Driving Distance - (ROUNDS),71.0
2,Bubba Watson,2010.0,Driving Distance,Driving Distance - (ROUNDS),77.0
3,Dustin Johnson,2010.0,Driving Distance,Driving Distance - (ROUNDS),83.0
4,Brett Wetterich,2010.0,Driving Distance,Driving Distance - (ROUNDS),54.0


In [14]:
pga_tour_data.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [33]:
pga_tour_data.describe(["Value"]).show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570728|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [34]:
pga_tour_data.select(['Value','Season']).summary('count', 'min','max').show()

+-------+-------+-------+
|summary|  Value| Season|
+-------+-------+-------+
|  count|1657247|2740403|
|    min|   -178|   2010|
|    max|3564954|   2018|
+-------+-------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [25]:
new_pga_data = pga_tour_data.select(['Season','Value'])
new_pga_data.limit(5).toPandas()

,Season,Value
0,NaN,NaN
1,2010.0,71.0
2,2010.0,77.0
3,2010.0,83.0
4,2010.0,54.0


In [26]:
new_pga_data.write.mode('overwrite').parquet('PGA_DATA_HW')

23/12/27 18:05:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/12/27 18:05:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/12/27 18:05:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/12/27 18:05:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/12/27 18:05:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/12/27 18:05:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/12/27 18:05:07 WARN MemoryManager: Total allocation exceeds 95.

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [27]:
new_pga_data.write.mode('overwrite').partitionBy('Season').parquet('PGA_DATA_SEASON_HW')

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [35]:
partitioned_data = spark.read.parquet('/Users/apple/Desktop/Learning_PySpark/PGA_DATA_HW/')

partitioned_data.limit(5).toPandas()

,Season,Value
0,2016,992.0
1,2016,NaN
2,2016,978.0
3,2016,794.0
4,2016,822.0


## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [39]:
subset_data = spark.read.option("basePath",'/Users/apple/Desktop/Learning_PySpark/PGA_DATA_SEASON_HW/').parquet(
                                 '/Users/apple/Desktop/Learning_PySpark/PGA_DATA_SEASON_HW/'+'Season=2010',\
                                 '/Users/apple/Desktop/Learning_PySpark/PGA_DATA_SEASON_HW/'+'Season=2011',\
                                 '/Users/apple/Desktop/Learning_PySpark/PGA_DATA_SEASON_HW/'+'Season=2012')

subset_data.show(5)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+
only showing top 5 rows



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [32]:
custom_data=[ ('Ram', 50, 'Maruti', 'Male'), ('Shyam', 50, 'Hyundai', 'Male'),('Seema', 25, 'Mahindra', 'Female'),
                                     ('Rati', 40, 'Tata', 'Female') ]
custom_df = spark.createDataFrame(custom_data,["Name", "Age", "Company", "Gender"])

custom_df.toPandas()

,Name,Age,Company,Gender
0,Ram,50,Maruti,Male
1,Shyam,50,Hyundai,Male
2,Seema,25,Mahindra,Female
3,Rati,40,Tata,Female


## We're done! Great job!